In [1]:
import numpy as np

In [2]:
import data_loading

In [3]:
percentile_rank_chars, regular_chars, chars, dates,\
    return_panel, permnos, rts, monthly_updates = data_loading.get_data_panel("../Data/raw_chars_returns_df_all_dates_yearly_fb.fthr",
                                                            "../Data/ff_rf.csv",
                                                                              computstat_data_present_filter=True,
                                                                              financial_firm_filter=False)

Index(['A2ME', 'AC', 'AT', 'ATO', 'B2M', 'BETA_d', 'BETA_m', 'C2A', 'CF2B',
       'CF2P', 'CTO', 'D2A', 'D2P', 'DPI2A', 'E2P', 'FC2Y', 'HIGH52', 'INV',
       'IdioVol', 'LEV', 'ME', 'NI', 'NOA', 'OA', 'OL', 'OP', 'PCM', 'PM',
       'PROF', 'Q', 'R12_2', 'R12_7', 'R2_1', 'R36_13', 'R60_13', 'RNA', 'ROA',
       'ROE', 'RVAR', 'S2P', 'SGA2S', 'SPREAD', 'SUV', 'TURN', 'VAR', 'return',
       'date', 'permno', 'monthly_update'],
      dtype='object')
['A2ME' 'AC' 'AT' 'ATO' 'B2M' 'BETA_d' 'BETA_m' 'C2A' 'CF2B' 'CF2P' 'CTO'
 'D2A' 'D2P' 'DPI2A' 'E2P' 'FC2Y' 'HIGH52' 'INV' 'IdioVol' 'LEV' 'ME' 'NI'
 'NOA' 'OA' 'OL' 'OP' 'PCM' 'PM' 'PROF' 'Q' 'R12_2' 'R12_7' 'R2_1'
 'R36_13' 'R60_13' 'RNA' 'ROA' 'ROE' 'RVAR' 'S2P' 'SGA2S' 'SPREAD' 'SUV'
 'TURN' 'VAR']


100%|████████████████████████████████████████████████████████████████████████████| 648/648 [03:06<00:00,  3.48it/s]


In [57]:
type(percentile_rank_chars)

numpy.ndarray

In [58]:
any_present = np.any(~np.isnan(percentile_rank_chars), axis=2)
present_counts = np.sum(np.sum(np.logical_and(~np.isnan(percentile_rank_chars),
                                             np.expand_dims(any_present, axis=2)), axis=0), axis=0)

In [62]:
ordering = np.argsort(present_counts)[::-1]

In [63]:
print(chars[ordering])
ordering = np.argsort(present_counts)[::-1]
permuted_chars = percentile_rank_chars[:,:,ordering]

['ME' 'R2_1' 'D2P' 'SPREAD' 'RVAR' 'VAR' 'IdioVol' 'TURN' 'SUV' 'R12_7'
 'CF2P' 'E2P' 'R12_2' 'S2P' 'B2M' 'CF2B' 'AT' 'A2ME' 'Q' 'PROF' 'PM' 'PCM'
 'BETA_m' 'C2A' 'OL' 'ROE' 'CTO' 'ATO' 'LEV' 'NOA' 'ROA' 'RNA' 'OP' 'NI'
 'INV' 'BETA_d' 'R36_13' 'D2A' 'FC2Y' 'SGA2S' 'OA' 'AC' 'HIGH52' 'R60_13'
 'DPI2A']


In [64]:
all_data = data_loading.get_data(permuted_chars*.99 + 0.5, 30, fill_val=-1)

100%|███████████████████████████████████████████████████████████████████████████| 648/648 [00:00<00:00, 691.01it/s]


In [42]:
import models, loss_functions

In [48]:
all_dataset = data_loading.ListDataset(all_data, batch_size=1)
train_data_loader = DataLoader(all_dataset, batch_size=100, shuffle=True, num_workers=4)

In [44]:
from torch.optim.lr_scheduler import MultiStepLR
import time
from tqdm.notebook import tqdm

In [45]:
import importlib

In [46]:
importlib.reload(models)

<module 'models' from '/home/selwin_p_george/CS 236 Project/Code/models.py'>

In [51]:
model = models.Beta_NADE(hidden_state_dim=6)

num_epochs = 21
eval_freq = 10

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


scheduler = MultiStepLR(optimizer, [20, 30], gamma=0.1, last_epoch=-1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f'torch device is {device}')
model = model.to(device)
model.sum_matrix = model.sum_matrix.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        model.batch_size = data[0].shape[0]
        assert not torch.isnan(data[0]).any()
        assert not torch.isnan(data[1]).any()
        C_train, C_mask = data
        C_train = C_train.squeeze(1)
        additional_state = torch.ones(C_train.shape[0], 6)
        model_input = torch.cat([additional_state, C_train], axis=1)
        model_input = model_input.float().to(device)
        

        log_likelihood = model(model_input)
        log_likelihood.mean().backward()

        losses.append(log_likelihood.data.cpu().numpy())
        
        if i % 10 == 0:
            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       
            
        
    print(f'epoch {epoch} avg loss {np.mean(losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()

torch device is cuda:0
epoch 0


  0%|          | 0/4535 [00:00<?, ?it/s]

torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51]

torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51,

torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51]

...

torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51]

torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51]

torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51]

torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51])
torch.Size([100, 51, 51])
torch.Size([100, 51, 5])
torch.Size([100, 45, 5])
torch.Size([100, 51]

KeyboardInterrupt: 

# XS + LSTM

In [1]:
import numpy as np
import data_loading
from tqdm.notebook import tqdm

In [2]:

percentile_rank_chars, regular_chars, chars, dates,\
    return_panel, permnos, rts, monthly_updates = data_loading.get_data_panel("../Data/raw_chars_returns_df_all_dates_yearly_fb.fthr",
                                                            "../Data/ff_rf.csv", computstat_data_present_filter=True,
                                                                              financial_firm_filter=False)

Index(['A2ME', 'AC', 'AT', 'ATO', 'B2M', 'BETA_d', 'BETA_m', 'C2A', 'CF2B',
       'CF2P', 'CTO', 'D2A', 'D2P', 'DPI2A', 'E2P', 'FC2Y', 'HIGH52', 'INV',
       'IdioVol', 'LEV', 'ME', 'NI', 'NOA', 'OA', 'OL', 'OP', 'PCM', 'PM',
       'PROF', 'Q', 'R12_2', 'R12_7', 'R2_1', 'R36_13', 'R60_13', 'RNA', 'ROA',
       'ROE', 'RVAR', 'S2P', 'SGA2S', 'SPREAD', 'SUV', 'TURN', 'VAR', 'return',
       'date', 'permno', 'monthly_update'],
      dtype='object')
['A2ME' 'AC' 'AT' 'ATO' 'B2M' 'BETA_d' 'BETA_m' 'C2A' 'CF2B' 'CF2P' 'CTO'
 'D2A' 'D2P' 'DPI2A' 'E2P' 'FC2Y' 'HIGH52' 'INV' 'IdioVol' 'LEV' 'ME' 'NI'
 'NOA' 'OA' 'OL' 'OP' 'PCM' 'PM' 'PROF' 'Q' 'R12_2' 'R12_7' 'R2_1'
 'R36_13' 'R60_13' 'RNA' 'ROA' 'ROE' 'RVAR' 'S2P' 'SGA2S' 'SPREAD' 'SUV'
 'TURN' 'VAR']


100%|████████████████████████████████████████████████████████████████████████████| 648/648 [03:05<00:00,  3.49it/s]


In [3]:
any_present = np.any(~np.isnan(percentile_rank_chars), axis=2)
present_counts = np.sum(np.sum(np.logical_and(~np.isnan(percentile_rank_chars),
                                             np.expand_dims(any_present, axis=2)), axis=0), axis=0)
del any_present

In [4]:
ordering = np.argsort(present_counts)[::-1]
print(chars[ordering])
percentile_rank_chars = percentile_rank_chars[:,:,ordering]

['ME' 'R2_1' 'D2P' 'SPREAD' 'RVAR' 'VAR' 'IdioVol' 'TURN' 'SUV' 'R12_7'
 'CF2P' 'E2P' 'R12_2' 'S2P' 'B2M' 'CF2B' 'AT' 'A2ME' 'Q' 'PROF' 'PM' 'PCM'
 'BETA_m' 'C2A' 'OL' 'ROE' 'CTO' 'ATO' 'LEV' 'NOA' 'ROA' 'RNA' 'OP' 'NI'
 'INV' 'BETA_d' 'R36_13' 'D2A' 'FC2Y' 'SGA2S' 'OA' 'AC' 'HIGH52' 'R60_13'
 'DPI2A']


In [5]:
gamma_ts = np.load('../Data/gamma_ts_save.npz')['data']

In [6]:
percentile_rank_chars.shape

(648, 22630, 45)

In [7]:
train_data = data_loading.get_data(percentile_rank_chars[:400], min_chars_present=30, fill_val=-1, 
                                   gamma_ts=gamma_ts[:400], ordered=True)

100%|████████████████████████████████████████████████████████████████████████████| 400/400 [00:06<00:00, 63.96it/s]


In [8]:
import importlib
importlib.reload(data_loading)

<module 'data_loading' from '/home/selwin_p_george/CS 236 Project/Code/data_loading.py'>

In [9]:
all_dataset = data_loading.ListDatasetWithFactors(train_data, batch_size=50)
train_data_loader = data_loading.DataLoader(all_dataset, batch_size=10, shuffle=True, num_workers=4)

In [10]:
import models, loss_functions

In [11]:
import numpy as np
import csv
import pandas as pd
import torch
import torch.nn as nn

In [12]:
import numpy as np
import csv
import pandas as pd
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR
import time

In [19]:
model = models.XSCharLSTM(input_dim=96, hidden_dim=6, num_layers=5, batch_size=10)
ar_model = models.Beta_NADE(hidden_state_dim=6)

num_epochs = 41
eval_freq = 10

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
ar_optimizer = torch.optim.Adam(ar_model.parameters(), lr=0.01)


scheduler = MultiStepLR(optimizer, [10, 20], gamma=0.1, last_epoch=-1)
ar_scheduler = MultiStepLR(ar_optimizer, [10, 20], gamma=0.1, last_epoch=-1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
ar_model = ar_model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    ar_losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        data, mask, ordered_mask, factors = data
        assert not torch.isnan(data).any()
#         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)
        
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        
      
        if train_input.shape[1] == 10:
            alpha_pred, beta_pred, hidden_out = model(train_input)
            loss = loss_fn(C_train[1:], alpha_pred[:-1], beta_pred[:-1], mask=C_mask[1:]).mean()
            assert not torch.isnan(loss).any()
            
            loss.backward()
            losses.append(loss.data.detach().cpu().numpy())
            
            model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
            batch_size, batch_length, dim = model_input.shape
            model_input = model_input.reshape(batch_size * batch_length, dim)
            
            
            
            C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
            C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

            alphas_pred, betas_pred = ar_model(model_input)
            
            
            ar_loss = loss_fn(C_train_reshape,
                              alphas_pred, betas_pred, mask=C_ordered_mask_reshape,
                             reduce_axis=1).mean()
            ar_loss.backward()
                        
#             for i in range(50):
#                 model_input = model_input[0:1]
#                 model_input.requires_grad_(True)
#                 alphas_pred, betas_pred = ar_model(model_input)
#                 res = torch.sum(input_mask * alphas_pred)
#                 print(res, res.shape, res.requires_grad)
#                 print(model_input.shape, model_input.requires_grad)
#                 input_mask = torch.zeros_like(alphas_pred).to(device)
#                 input_mask[0,i] = 1
#                 print(i, torch.autograd.grad(res, model_input, create_graph=True))
            
            ar_losses.append(ar_loss.data.detach().cpu().numpy())
            
            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       

            ar_optimizer.step()
            ar_model.zero_grad()
            ar_optimizer.zero_grad()

    print(f'epoch {epoch} avg lstm loss {np.mean(losses)}, avg ar model loss {np.mean(ar_losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    ar_scheduler.step()

Parameter containing:
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.]], requires_grad=True) torch.Size([45, 51])
torch device is cuda:0
epoch 0


  0%|          | 0/698 [00:00<?, ?it/s]

epoch 0 avg lstm loss 49.45719909667969, avg ar model loss 378.0188293457031
epoch 0 took 0.4881410797437032 minutes
epoch 1


  0%|          | 0/698 [00:00<?, ?it/s]

epoch 1 avg lstm loss -0.02864704094827175, avg ar model loss 20.501192092895508
epoch 1 took 0.48964365720748904 minutes
epoch 2


...

epoch 3 avg lstm loss 0.009909706190228462, avg ar model loss -0.40719810128211975
epoch 3 took 0.48851577043533323 minutes
epoch 4


  0%|          | 0/698 [00:00<?, ?it/s]

epoch 4 avg lstm loss 0.02319185622036457, avg ar model loss -0.583076536655426
epoch 4 took 0.48716028531392414 minutes
epoch 5


  0%|          | 0/698 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         print(data, mask)
    assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)

    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)


    if train_input.shape[1] == 10:
        alpha_pred, beta_pred, hidden_out = model(train_input)

        model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
        batch_size, batch_length, dim = model_input.shape
        model_input = model_input.reshape(batch_size * batch_length, dim)

        C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
        C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

        alphas_pred, betas_pred = ar_model(model_input)
        pred = alphas_pred / (alphas_pred + betas_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train_reshape.detach().cpu().numpy())
        train_masks.append(C_ordered_mask_reshape.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan
mse = np.nanmean((preds - gts)**2, axis=0)
list(zip(chars[ordering], np.sqrt(mse)))

  0%|          | 0/698 [00:00<?, ?it/s]

[('ME', 0.016994426),
 ('R2_1', 0.018275818),
 ('D2P', 0.011704876),
 ('SPREAD', 0.010778561),
 ('RVAR', 0.0145637365),
 ('VAR', 0.013189538),
 ('IdioVol', 0.01024529),
 ('TURN', 0.010119821),
 ('SUV', 0.008239745),
 ('R12_7', 0.014688572),
 ('CF2P', 0.012033416),
 ('E2P', 0.011507633),
 ('R12_2', 0.014200185),
 ('S2P', 0.015179531),
 ('B2M', 0.021411572),
 ('CF2B', 0.014513005),
 ('AT', 0.017653378),
 ('A2ME', 0.015375111),
 ('Q', 0.014285593),
 ('PROF', 0.00805442),
 ('PM', 0.010016139),
 ('PCM', 0.00839753),
 ('BETA_m', 0.014789646),
 ('C2A', 0.01997428),
 ('OL', 0.021216303),
 ('ROE', 0.007967139),
 ('CTO', 0.01942396),
 ('ATO', 0.011658184),
 ('LEV', 0.011625119),
 ('NOA', 0.02127807),
 ('ROA', 0.01048511),
 ('RNA', 0.007962812),
 ('OP', 0.010118417),
 ('NI', 0.01209986),
 ('INV', 0.008693303),
 ('BETA_d', 0.018214818),
 ('R36_13', 0.01655251),
 ('D2A', 0.01572101),
 ('FC2Y', 0.009094659),
 ('SGA2S', 0.009705063),
 ('OA', 0.013145341),
 ('AC', 0.014749289),
 ('HIGH52', 0.014553352

In [ ]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         assert not (data <= 0).any(), torch.min(data)
#         print(data.shape)
    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)       
    if train_input.shape[1] == 10:
        alpha_pred, beta_pred,_ = model(train_input)

        pred = alpha_pred / (alpha_pred + beta_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train.detach().cpu().numpy())
        train_masks.append(C_mask.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan

mse = np.nanmean((preds - gts)**2, axis=(0,1))
list(zip(chars[ordering], np.sqrt(mse))), np.mean(np.sqrt(mse))

In [39]:
val_data = data_loading.get_data(percentile_rank_chars[400:600], min_chars_present=30, fill_val=-1, 
                                   gamma_ts=gamma_ts[400:600], ordered=True)

100%|████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 52.36it/s]


In [40]:
all_dataset = data_loading.ListDatasetWithFactors(val_data, batch_size=50)
train_data_loader = data_loading.DataLoader(all_dataset, batch_size=10, shuffle=True, num_workers=4)

In [41]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         print(data, mask)
    assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)

    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)


    if train_input.shape[1] == 10:
        alpha_pred, beta_pred, hidden_out = model(train_input)

        model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
        batch_size, batch_length, dim = model_input.shape
        model_input = model_input.reshape(batch_size * batch_length, dim)

        C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
        C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

        alphas_pred, betas_pred = ar_model(model_input)
        pred = alphas_pred / (alphas_pred + betas_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train_reshape.detach().cpu().numpy())
        train_masks.append(C_ordered_mask_reshape.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan
mse = np.nanmean((preds - gts)**2, axis=0)
list(zip(chars[ordering], np.sqrt(mse)))

  0%|          | 0/475 [00:00<?, ?it/s]

[('ME', 0.009420439),
 ('R2_1', 0.015046226),
 ('D2P', 0.012610359),
 ('SPREAD', 0.0109960325),
 ('RVAR', 0.022174569),
 ('VAR', 0.010580117),
 ('IdioVol', 0.009986096),
 ('TURN', 0.006420104),
 ('SUV', 0.0069246166),
 ('R12_7', 0.009092975),
 ('CF2P', 0.0066264374),
 ('E2P', 0.011351572),
 ('R12_2', 0.005376908),
 ('S2P', 0.0128406165),
 ('B2M', 0.006662936),
 ('CF2B', 0.008431717),
 ('AT', 0.006823305),
 ('A2ME', 0.019575287),
 ('Q', 0.01590447),
 ('PROF', 0.005817745),
 ('PM', 0.006029416),
 ('PCM', 0.014015055),
 ('BETA_m', 0.008359386),
 ('C2A', 0.00525353),
 ('OL', 0.013458848),
 ('ROE', 0.005667036),
 ('CTO', 0.009215627),
 ('ATO', 0.0064174696),
 ('LEV', 0.014428355),
 ('NOA', 0.011127448),
 ('ROA', 0.011565131),
 ('RNA', 0.010131157),
 ('OP', 0.0092309415),
 ('NI', 0.0113786645),
 ('INV', 0.014147015),
 ('BETA_d', 0.0074790884),
 ('R36_13', 0.010916909),
 ('D2A', 0.013918674),
 ('FC2Y', 0.0090912925),
 ('SGA2S', 0.015098919),
 ('OA', 0.013236991),
 ('AC', 0.0057610646),
 ('HIG

In [42]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         assert not (data <= 0).any(), torch.min(data)
#         print(data.shape)
    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)       
    if train_input.shape[1] == 10:
        alpha_pred, beta_pred,_ = model(train_input)

        pred = alpha_pred / (alpha_pred + beta_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train.detach().cpu().numpy())
        train_masks.append(C_mask.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan

mse = np.nanmean((preds - gts)**2, axis=(0,1))
list(zip(chars[ordering], np.sqrt(mse))), np.mean(np.sqrt(mse))

  0%|          | 0/475 [00:00<?, ?it/s]

In [43]:
list(zip(chars[ordering], np.sqrt(mse))), np.mean(np.sqrt(mse))

([('ME', 0.2882925),
  ('R2_1', 0.2861115),
  ('D2P', 0.29338998),
  ('SPREAD', 0.28889006),
  ('RVAR', 0.27280915),
  ('VAR', 0.2739712),
  ('IdioVol', 0.27303356),
  ('TURN', 0.28869623),
  ('SUV', 0.2847902),
  ('R12_7', 0.28743878),
  ('CF2P', 0.28312084),
  ('E2P', 0.28100437),
  ('R12_2', 0.28635156),
  ('S2P', 0.28011435),
  ('B2M', 0.28301242),
  ('CF2B', 0.28014743),
  ('AT', 0.28357416),
  ('A2ME', 0.2842634),
  ('Q', 0.2837084),
  ('PROF', 0.28037572),
  ('PM', 0.27831548),
  ('PCM', 0.28394076),
  ('BETA_m', 0.29008412),
  ('C2A', 0.28667018),
  ('OL', 0.28245237),
  ('ROE', 0.28184167),
  ('CTO', 0.28100416),
  ('ATO', 0.28132066),
  ('LEV', 0.28749672),
  ('NOA', 0.2832554),
  ('ROA', 0.2837932),
  ('RNA', 0.28129646),
  ('OP', 0.27933842),
  ('NI', 0.28610462),
  ('INV', 0.28695932),
  ('BETA_d', 0.29208195),
  ('R36_13', 0.29069445),
  ('D2A', 0.28611156),
  ('FC2Y', 0.2788018),
  ('SGA2S', 0.2795629),
  ('OA', 0.2829811),
  ('AC', 0.28346103),
  ('HIGH52', 0.28267318),

# Optimize Jointly

In [20]:
train_data = data_loading.get_data(percentile_rank_chars[:400], min_chars_present=30, fill_val=-1, 
                                   gamma_ts=gamma_ts[:400], ordered=True)
all_dataset = data_loading.ListDatasetWithFactors(train_data, batch_size=50)
train_data_loader = data_loading.DataLoader(all_dataset, batch_size=10, shuffle=True, num_workers=4)

100%|████████████████████████████████████████████████████████████████████████████| 400/400 [00:06<00:00, 64.01it/s]


In [21]:
# model = models.XSCharLSTM(input_dim=96, hiddin_dim=15, num_layers=2, batch_size=10)
# ar_model = models.Beta_NADE(hidden_state_dim=6)

num_epochs = 41
eval_freq = 10

optimizer = torch.optim.Adam(list(model.parameters()) + list(ar_model.parameters()), lr=0.01)


scheduler = MultiStepLR(optimizer, [20, 35], gamma=0.1, last_epoch=-1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
ar_model = ar_model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    ar_losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        data, mask, ordered_mask, factors = data
        assert not torch.isnan(data).any()
#         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)
        
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        
      
        if train_input.shape[1] == 10:
            alpha_pred, beta_pred, hidden_out = model(train_input)
            loss = loss_fn(C_train[1:], alpha_pred[:-1], beta_pred[:-1], mask=C_mask[1:]).mean()
            assert not torch.isnan(loss).any()
            
#             loss.backward()
            losses.append(loss.data.detach().cpu().numpy())
            
#             print(hidden_out.shape, C_train.shape)
            model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
            batch_size, batch_length, dim = model_input.shape
            model_input = model_input.reshape(batch_size * batch_length, dim)
            
            C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
            C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

            alphas_pred, betas_pred = ar_model(model_input)
            
            ar_loss = loss_fn(C_train_reshape,
                              alphas_pred, betas_pred, mask=C_ordered_mask_reshape,
                             reduce_axis=1).mean()
            
            net_loss = ar_loss + loss
            net_loss.backward()
            ar_losses.append(ar_loss.data.detach().cpu().numpy())
            
#             if i % 100 == 0:
#                 print(loss)
#                 print(ar_loss)
            
#             if i % 10 == 0:
    #             torch.nn.utils.clip_grad_norm_(model.parameters(), 50)
            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       
                

    print(f'epoch {epoch} avg lstm loss {np.mean(losses)}, avg ar model loss {np.mean(ar_losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    ar_scheduler.step()

torch device is cuda:0
epoch 0


  0%|          | 0/698 [00:00<?, ?it/s]

epoch 0 avg lstm loss 0.09326416999101639, avg ar model loss -41.55487060546875
epoch 0 took 0.4808860143025716 minutes
epoch 1


  0%|          | 0/698 [00:00<?, ?it/s]

epoch 1 avg lstm loss 0.07046534866094589, avg ar model loss -82.2635726928711
epoch 1 took 0.4803778290748596 minutes
epoch 2


...

epoch 15 avg lstm loss -9.179129600524902, avg ar model loss -98.66886138916016
epoch 15 took 0.4796007434527079 minutes
epoch 16


  0%|          | 0/698 [00:00<?, ?it/s]

epoch 16 avg lstm loss -9.333342552185059, avg ar model loss -98.76403045654297
epoch 16 took 0.4813433329264323 minutes
epoch 17


  0%|          | 0/698 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         print(data, mask)
    assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)

    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)


    if train_input.shape[1] == 10:
        alpha_pred, beta_pred, hidden_out = model(train_input)

        model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
        batch_size, batch_length, dim = model_input.shape
        model_input = model_input.reshape(batch_size * batch_length, dim)

        C_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
        C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
        C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

        alphas_pred, betas_pred = ar_model(model_input)
        pred = alphas_pred / (alphas_pred + betas_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train_reshape.detach().cpu().numpy())
        train_masks.append(C_ordered_mask_reshape.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan
mse = np.nanmean((preds - gts)**2, axis=0)
list(zip(chars[ordering], np.sqrt(mse)))

  0%|          | 0/698 [00:00<?, ?it/s]

[('ME', 0.007981713),
 ('R2_1', 0.006885205),
 ('D2P', 0.0118199615),
 ('SPREAD', 0.0062841927),
 ('RVAR', 0.0074379155),
 ('VAR', 0.007371009),
 ('IdioVol', 0.0070895418),
 ('TURN', 0.005612313),
 ('SUV', 0.006341484),
 ('R12_7', 0.007576596),
 ('CF2P', 0.009736982),
 ('E2P', 0.008215547),
 ('R12_2', 0.0061396547),
 ('S2P', 0.012034022),
 ('B2M', 0.015507225),
 ('CF2B', 0.0064073396),
 ('AT', 0.014131421),
 ('A2ME', 0.011805018),
 ('Q', 0.007177227),
 ('PROF', 0.007877403),
 ('PM', 0.009971296),
 ('PCM', 0.007381135),
 ('BETA_m', 0.011924914),
 ('C2A', 0.0092319865),
 ('OL', 0.01362965),
 ('ROE', 0.0068265377),
 ('CTO', 0.008582859),
 ('ATO', 0.009116479),
 ('LEV', 0.016384605),
 ('NOA', 0.0073727127),
 ('ROA', 0.007902001),
 ('RNA', 0.0076342486),
 ('OP', 0.0069478597),
 ('NI', 0.009699914),
 ('INV', 0.0058400277),
 ('BETA_d', 0.010533223),
 ('R36_13', 0.0060214708),
 ('D2A', 0.008907987),
 ('FC2Y', 0.019286048),
 ('SGA2S', 0.014578915),
 ('OA', 0.0082325665),
 ('AC', 0.0076015987),


In [23]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         assert not (data <= 0).any(), torch.min(data)
#         print(data.shape)
    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)       
    if train_input.shape[1] == 10:
        alpha_pred, beta_pred,_ = model(train_input)

        pred = alpha_pred / (alpha_pred + beta_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train.detach().cpu().numpy())
        train_masks.append(C_mask.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan

mse = np.nanmean((preds - gts)**2, axis=(0,1))
list(zip(chars[ordering], np.sqrt(mse))), np.mean(np.sqrt(mse))

  0%|          | 0/698 [00:00<?, ?it/s]

([('ME', 0.20989154),
  ('R2_1', 0.27405956),
  ('D2P', 0.24115486),
  ('SPREAD', 0.1802615),
  ('RVAR', 0.12719882),
  ('VAR', 0.13032171),
  ('IdioVol', 0.13907516),
  ('TURN', 0.26824632),
  ('SUV', 0.2846091),
  ('R12_7', 0.26243508),
  ('CF2P', 0.236937),
  ('E2P', 0.25259954),
  ('R12_2', 0.25672165),
  ('S2P', 0.20594539),
  ('B2M', 0.18664782),
  ('CF2B', 0.2703901),
  ('AT', 0.19596305),
  ('A2ME', 0.07628629),
  ('Q', 0.070202984),
  ('PROF', 0.27266416),
  ('PM', 0.23614624),
  ('PCM', 0.2677706),
  ('BETA_m', 0.2596293),
  ('C2A', 0.26492605),
  ('OL', 0.27337477),
  ('ROE', 0.24692792),
  ('CTO', 0.27201715),
  ('ATO', 0.2701464),
  ('LEV', 0.23253366),
  ('NOA', 0.2752484),
  ('ROA', 0.22761074),
  ('RNA', 0.23910455),
  ('OP', 0.25431982),
  ('NI', 0.28108224),
  ('INV', 0.26003313),
  ('BETA_d', 0.2605864),
  ('R36_13', 0.26294377),
  ('D2A', 0.2687445),
  ('FC2Y', 0.25788352),
  ('SGA2S', 0.25555232),
  ('OA', 0.2774026),
  ('AC', 0.2780474),
  ('HIGH52', 0.2423506),
 

In [24]:
val_data = data_loading.get_data(percentile_rank_chars[400:600], min_chars_present=30, fill_val=-1, 
                                   gamma_ts=gamma_ts[400:600], ordered=True)
all_dataset = data_loading.ListDatasetWithFactors(val_data, batch_size=50)
train_data_loader = data_loading.DataLoader(all_dataset, batch_size=10, shuffle=True, num_workers=4)

100%|████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 46.74it/s]


In [27]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         print(data, mask)
    assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)

    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)


    if train_input.shape[1] == 10:
        alpha_pred, beta_pred, hidden_out = model(train_input)

        model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
        batch_size, batch_length, dim = model_input.shape
        model_input = model_input.reshape(batch_size * batch_length, dim)

        C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
        C_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
        C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

        alphas_pred, betas_pred = ar_model(model_input)
        pred = alphas_pred / (alphas_pred + betas_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train_reshape.detach().cpu().numpy())
        train_masks.append(C_mask_reshape.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan
mse = np.nanmean((preds - gts)**2, axis=0)
list(zip(chars[ordering], np.sqrt(mse)))

  0%|          | 0/475 [00:00<?, ?it/s]

[('ME', 0.007650777),
 ('R2_1', 0.0068526855),
 ('D2P', 0.011803265),
 ('SPREAD', 0.0063405163),
 ('RVAR', 0.008038786),
 ('VAR', 0.007911834),
 ('IdioVol', 0.007604321),
 ('TURN', 0.0056969062),
 ('SUV', 0.006390973),
 ('R12_7', 0.0075663207),
 ('CF2P', 0.010530385),
 ('E2P', 0.00801664),
 ('R12_2', 0.006361285),
 ('S2P', 0.011934971),
 ('B2M', 0.015291166),
 ('CF2B', 0.0063031167),
 ('AT', 0.014312125),
 ('A2ME', 0.011729534),
 ('Q', 0.0074521448),
 ('PROF', 0.007867415),
 ('PM', 0.0096683195),
 ('PCM', 0.007961408),
 ('BETA_m', 0.012328836),
 ('C2A', 0.00973881),
 ('OL', 0.014086157),
 ('ROE', 0.006819704),
 ('CTO', 0.0088106245),
 ('ATO', 0.008869508),
 ('LEV', 0.016589658),
 ('NOA', 0.007395937),
 ('ROA', 0.00800197),
 ('RNA', 0.007928216),
 ('OP', 0.007415824),
 ('NI', 0.009967804),
 ('INV', 0.0060835853),
 ('BETA_d', 0.010645336),
 ('R36_13', 0.0062357564),
 ('D2A', 0.009873238),
 ('FC2Y', 0.019410893),
 ('SGA2S', 0.015107715),
 ('OA', 0.00930093),
 ('AC', 0.008188342),
 ('HIGH5

In [28]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         assert not (data <= 0).any(), torch.min(data)
#         print(data.shape)
    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)       
    if train_input.shape[1] == 10:
        alpha_pred, beta_pred,_ = model(train_input)

        pred = alpha_pred / (alpha_pred + beta_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train.detach().cpu().numpy())
        train_masks.append(C_mask.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan

mse = np.nanmean((preds - gts)**2, axis=(0,1))
list(zip(chars[ordering], np.sqrt(mse))), np.mean(np.sqrt(mse))

  0%|          | 0/475 [00:00<?, ?it/s]

([('ME', 0.2270785),
  ('R2_1', 0.28524897),
  ('D2P', 0.255948),
  ('SPREAD', 0.1669922),
  ('RVAR', 0.13371357),
  ('VAR', 0.14064394),
  ('IdioVol', 0.14511827),
  ('TURN', 0.2733414),
  ('SUV', 0.28532574),
  ('R12_7', 0.2755995),
  ('CF2P', 0.23992816),
  ('E2P', 0.24618544),
  ('R12_2', 0.26989412),
  ('S2P', 0.23038886),
  ('B2M', 0.19872694),
  ('CF2B', 0.26974013),
  ('AT', 0.2100129),
  ('A2ME', 0.076190904),
  ('Q', 0.07180035),
  ('PROF', 0.28141898),
  ('PM', 0.23164721),
  ('PCM', 0.2809683),
  ('BETA_m', 0.26011577),
  ('C2A', 0.2597642),
  ('OL', 0.2681491),
  ('ROE', 0.25607952),
  ('CTO', 0.27569234),
  ('ATO', 0.27708998),
  ('LEV', 0.24328746),
  ('NOA', 0.2903115),
  ('ROA', 0.26006886),
  ('RNA', 0.25442174),
  ('OP', 0.25949022),
  ('NI', 0.27820268),
  ('INV', 0.27399093),
  ('BETA_d', 0.27885318),
  ('R36_13', 0.27320355),
  ('D2A', 0.2738948),
  ('FC2Y', 0.2561601),
  ('SGA2S', 0.25752795),
  ('OA', 0.28348666),
  ('AC', 0.28339788),
  ('HIGH52', 0.23889722),


# OOS 

In [1]:
print

<function print>

In [2]:
import numpy as np
import data_loading
from tqdm.notebook import tqdm

In [3]:

percentile_rank_chars, _, chars, _,\
    _, _, _, monthly_updates = data_loading.get_data_panel("../Data/raw_chars_returns_df_all_dates_yearly_fb.fthr",
                                                            "../Data/ff_rf.csv", computstat_data_present_filter=True,
                                                                              financial_firm_filter=False)

Index(['A2ME', 'AC', 'AT', 'ATO', 'B2M', 'BETA_d', 'BETA_m', 'C2A', 'CF2B',
       'CF2P', 'CTO', 'D2A', 'D2P', 'DPI2A', 'E2P', 'FC2Y', 'HIGH52', 'INV',
       'IdioVol', 'LEV', 'ME', 'NI', 'NOA', 'OA', 'OL', 'OP', 'PCM', 'PM',
       'PROF', 'Q', 'R12_2', 'R12_7', 'R2_1', 'R36_13', 'R60_13', 'RNA', 'ROA',
       'ROE', 'RVAR', 'S2P', 'SGA2S', 'SPREAD', 'SUV', 'TURN', 'VAR', 'return',
       'date', 'permno', 'monthly_update'],
      dtype='object')
['A2ME' 'AC' 'AT' 'ATO' 'B2M' 'BETA_d' 'BETA_m' 'C2A' 'CF2B' 'CF2P' 'CTO'
 'D2A' 'D2P' 'DPI2A' 'E2P' 'FC2Y' 'HIGH52' 'INV' 'IdioVol' 'LEV' 'ME' 'NI'
 'NOA' 'OA' 'OL' 'OP' 'PCM' 'PM' 'PROF' 'Q' 'R12_2' 'R12_7' 'R2_1'
 'R36_13' 'R60_13' 'RNA' 'ROA' 'ROE' 'RVAR' 'S2P' 'SGA2S' 'SPREAD' 'SUV'
 'TURN' 'VAR']


100%|████████████████████████████████████████████████████████████████████████████| 648/648 [02:58<00:00,  3.63it/s]


In [4]:
any_present = np.any(~np.isnan(percentile_rank_chars), axis=2)
present_counts = np.sum(np.sum(np.logical_and(~np.isnan(percentile_rank_chars),
                                             np.expand_dims(any_present, axis=2)), axis=0), axis=0)
del any_present

In [5]:
ordering = np.argsort(present_counts)[::-1]
print(chars[ordering])
percentile_rank_chars = percentile_rank_chars[:,:,ordering]
chars = chars[ordering]

['ME' 'R2_1' 'D2P' 'SPREAD' 'RVAR' 'VAR' 'IdioVol' 'TURN' 'SUV' 'R12_7'
 'CF2P' 'E2P' 'R12_2' 'S2P' 'B2M' 'CF2B' 'AT' 'A2ME' 'Q' 'PROF' 'PM' 'PCM'
 'BETA_m' 'C2A' 'OL' 'ROE' 'CTO' 'ATO' 'LEV' 'NOA' 'ROA' 'RNA' 'OP' 'NI'
 'INV' 'BETA_d' 'R36_13' 'D2A' 'FC2Y' 'SGA2S' 'OA' 'AC' 'HIGH52' 'R60_13'
 'DPI2A']


In [6]:
char_groupings  = [('A2ME', "Q"),
                   ('AC', 'Q'),
('AT', 'Q'),
('ATO', 'Q'),
('B2M', 'QM'),
('BETA_d', 'M'),
('BETA_m', 'M'),
('C2A', 'Q'),
('CF2B', 'Q'),
('CF2P', 'QM'),
('CTO', 'Q'),
('D2A', 'Q'),
('D2P', 'M'),
('DPI2A', 'Q'),
('E2P', 'QM'),
('FC2Y', 'QY'),
('IdioVol', 'M'),
('INV', 'Q'),
('LEV', 'Q'),
('ME', 'M'),
('TURN', 'M'),
('NI', 'Q'),
('NOA', 'Q'),
('OA', 'Q'),
('OL', 'Q'),
('OP', 'Q'),
('PCM', 'Q'),
('PM', 'Q'),
('PROF', 'QY'),
('Q', 'QM'),
('R2_1', 'M'),
('R12_2', 'M'),
('R12_7', 'M'),
('R36_13', 'M'),
('R60_13', 'M'),
('HIGH52', 'M'),
('RVAR', 'M'),
('RNA', 'Q'),
('ROA', 'Q'),
('ROE', 'Q'),
('S2P', 'QM'),
('SGA2S', 'Q'),
('SPREAD', 'M'),
('SUV', 'M'),
('VAR', 'M')]
char_map = {x[0]:x[1] for x in char_groupings}

In [7]:
def load_imputation(name, full=False):
    base_path = '../Data/'
    result_file_name = base_path + name + '.npz'
    res = np.load(result_file_name)
    if not full:
        return res['data']
    else:
        return res['data'], res['dates'], res['permnos'], res['chars']

In [8]:
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [11]:
to_replace = np.isnan(masked_data[:400])
masked_data[:400][to_replace] = percentile_rank_chars[:400][to_replace]

In [10]:
masked_data = load_imputation('MAR_fit_data')[:, :, ordering]
print("loaded")
eval_data = load_imputation('MAR_eval_data')[400:, :, ordering]
print("loaded")
gamma_ts = load_imputation('MAR_xs_factors')
print("loaded")

loaded
loaded
loaded


In [12]:
train_gamma_ts = np.load('../Data/gamma_ts_save.npz')['data']

In [14]:
train_data_seq = data_loading.get_data(percentile_rank_chars[:400], min_chars_present=20, fill_val=-1, 
                                   gamma_ts=train_gamma_ts[:400], ordered=True)

100%|████████████████████████████████████████████████████████████████████████████| 400/400 [00:07<00:00, 55.52it/s]


In [16]:
len(train_data_seq)

8770

In [15]:
eval_data_seq = data_loading.get_data(masked_data[400:500], min_chars_present=20, fill_val=-1, 
                                   gamma_ts=gamma_ts[400:500], ordered=False,
                    oos_mask=~np.isnan(eval_data[:100]), oos_eval_data=eval_data[:100])
test_data_seq = data_loading.get_data(masked_data[500:], min_chars_present=20, fill_val=-1, 
                                   gamma_ts=gamma_ts[500:], ordered=False,
                    oos_mask=~np.isnan(eval_data[100:]), oos_eval_data=eval_data[100:])

100%|████████████████████████████████████████████████████████████████████████████| 148/148 [00:02<00:00, 60.93it/s]


In [17]:
len(eval_data_seq), len(test_data_seq)

(3159, 3220)

In [18]:
import importlib
importlib.reload(data_loading)

<module 'data_loading' from '/home/selwin_p_george/CS 236 Project/Code/data_loading.py'>

In [19]:
all_dataset = data_loading.ListDatasetWithFactors(train_data_seq, batch_size=50)
train_data_loader = data_loading.DataLoader(all_dataset, batch_size=10, shuffle=True, num_workers=4)

all_dataset = data_loading.ListDatasetWithFactors(eval_data_seq, batch_size=50,
                                                 return_oos_data=True)
eval_data_loader = data_loading.DataLoader(all_dataset, batch_size=10, shuffle=True, num_workers=4)

all_dataset = data_loading.ListDatasetWithFactors(test_data_seq, batch_size=50,
                                                 return_oos_data=True)
test_data_loader = data_loading.DataLoader(all_dataset, batch_size=10, shuffle=True, num_workers=4)


In [20]:
import models
import loss_functions

In [21]:
import numpy as np
import csv
import pandas as pd
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR
import time

In [22]:
importlib.reload(models)

<module 'models' from '/home/selwin_p_george/CS 236 Project/Code/models.py'>

In [77]:
model = models.XSCharLSTM(input_dim=96, hiddin_dim=15, num_layers=2, batch_size=10)
ar_model = models.Beta_NADE(hidden_state_dim=6)

num_epochs = 41
eval_freq = 10

optimizer = torch.optim.Adam(list(model.parameters()) + list(ar_model.parameters()), lr=0.01)


scheduler = MultiStepLR(optimizer, [20, 35], gamma=0.1, last_epoch=-1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
ar_model = ar_model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    ar_losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        data, mask, ordered_mask, factors = data
        assert not torch.isnan(data).any()
#         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)
        
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        
      
        if train_input.shape[1] == 10:
            alpha_pred, beta_pred, hidden_out = model(train_input)
            loss = loss_fn(C_train[1:], alpha_pred[:-1], beta_pred[:-1], mask=C_mask[1:]).mean()
            assert not torch.isnan(loss).any()
            
#             loss.backward()
            losses.append(loss.data.detach().cpu().numpy())
            
#             print(hidden_out.shape, C_train.shape)
            model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
            batch_size, batch_length, dim = model_input.shape
            model_input = model_input.reshape(batch_size * batch_length, dim)
            
            C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
            C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

            alphas_pred, betas_pred = ar_model(model_input)
            
            ar_loss = loss_fn(C_train_reshape,
                              alphas_pred, betas_pred, mask=C_ordered_mask_reshape,
                             reduce_axis=1).mean()
            
            net_loss = ar_loss + loss
            net_loss.backward()
            ar_losses.append(ar_loss.data.detach().cpu().numpy())
            
#             if i % 100 == 0:
#                 print(loss)
#                 print(ar_loss)
            
#             if i % 10 == 0:
    #             torch.nn.utils.clip_grad_norm_(model.parameters(), 50)
            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       

    print(f'epoch {epoch} avg lstm loss {np.mean(losses)}, avg ar model loss {np.mean(ar_losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    if (epoch + 1) % 5 == 0 :
        if epoch > 5:
            print(eval_model_is(model, ar_model, train_data_loader))
        print(eval_model_oos(model, ar_model, eval_data_loader))
        print(eval_ts_model_oos(model, eval_data_loader))

Parameter containing:
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.]], requires_grad=True) torch.Size([45, 51])
torch device is cuda:0
epoch 0


  0%|          | 0/1201 [00:00<?, ?it/s]

epoch 0 avg lstm loss 21.975215911865234, avg ar model loss 185.47482299804688
epoch 0 took 0.6181668559710185 minutes
epoch 1


  0%|          | 0/1201 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# model = models.XSCharLSTM(input_dim=96, hidden_dim=15, num_layers=2, batch_size=10)

num_epochs = 41
eval_freq = 10

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


scheduler = MultiStepLR(optimizer, [20, 30], gamma=0.1, last_epoch=-1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        data, mask, ordered_mask, factors = data
        assert not torch.isnan(data).any()
#         assert not (data <= 0).any(), torch.min(data)
#         print(data.shape)
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        
        if train_input.shape[1] == 10 and train_input.shape[2] > 0:
#             print(train_input.shape)
            alpha_pred, beta_pred, hidden_out = model(train_input)
#             print(hidden_out.shape)

            loss = loss_fn(C_train[1:], alpha_pred[:-1], beta_pred[:-1], mask=C_mask[1:]).mean()
            assert not torch.isnan(loss).any()

            loss.backward()
            losses.append(loss.data.cpu().detach().numpy())

            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       


    print(f'epoch {epoch} avg loss {np.mean(losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    
    if (epoch + 1) % 5 == 0 :
        print(eval_ts_model_is(model, train_data_loader))
        print(eval_ts_model_is(model, eval_data_loader))
        print(eval_ts_model_oos(model, eval_data_loader))
        

torch device is cuda:0
epoch 0


  0%|          | 0/867 [00:00<?, ?it/s]

epoch 0 avg loss -27.41938591003418
epoch 0 took 0.2311870018641154 minutes
epoch 1


  0%|          | 0/867 [00:00<?, ?it/s]

epoch 1 avg loss -28.285863876342773
epoch 1 took 0.22755613327026367 minutes
epoch 2


...

  0%|          | 0/867 [00:00<?, ?it/s]

epoch 6 avg loss -30.217166900634766
epoch 6 took 0.23060224453608194 minutes
epoch 7


  0%|          | 0/867 [00:00<?, ?it/s]

epoch 7 avg loss -30.455570220947266
epoch 7 took 0.22894989252090453 minutes
epoch 8


  0%|          | 0/867 [00:00<?, ?it/s]

In [105]:
model = models.XSCharLSTM(input_dim=96, hidden_dim=6, num_layers=2, batch_size=10)
ar_model = models.Beta_NADE(hidden_state_dim=6)

num_epochs = 41
eval_freq = 10

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
ar_optimizer = torch.optim.Adam(ar_model.parameters(), lr=0.01)


scheduler = MultiStepLR(optimizer, [10, 20], gamma=0.1, last_epoch=-1)
ar_scheduler = MultiStepLR(ar_optimizer, [10, 20], gamma=0.1, last_epoch=-1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
ar_model = ar_model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    ar_losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        data, mask, ordered_mask, factors = data
        assert not torch.isnan(data).any()
#         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)
        
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        
      
        if train_input.shape[1] == 10:
            alpha_pred, beta_pred, hidden_out = model(train_input)
            loss = loss_fn(C_train[1:], alpha_pred[:-1], beta_pred[:-1], mask=C_mask[1:]).mean()
            assert not torch.isnan(loss).any()
            
            loss.backward()
            losses.append(loss.data.detach().cpu().numpy())
            
            model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
            batch_size, batch_length, dim = model_input.shape
            model_input = model_input.reshape(batch_size * batch_length, dim)
            
            
            
            C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
            C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

            alphas_pred, betas_pred = ar_model(model_input)
            
            
            ar_loss = loss_fn(C_train_reshape,
                              alphas_pred, betas_pred, mask=C_ordered_mask_reshape,
                             reduce_axis=1).mean()
            ar_loss.backward()
                        
#             for i in range(50):
#                 model_input = model_input[0:1]
#                 model_input.requires_grad_(True)
#                 alphas_pred, betas_pred = ar_model(model_input)
#                 res = torch.sum(input_mask * alphas_pred)
#                 print(res, res.shape, res.requires_grad)
#                 print(model_input.shape, model_input.requires_grad)
#                 input_mask = torch.zeros_like(alphas_pred).to(device)
#                 input_mask[0,i] = 1
#                 print(i, torch.autograd.grad(res, model_input, create_graph=True))
            
            ar_losses.append(ar_loss.data.detach().cpu().numpy())
            
            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       

            ar_optimizer.step()
            ar_model.zero_grad()
            ar_optimizer.zero_grad()

    print(f'epoch {epoch} avg lstm loss {np.mean(losses)}, avg ar model loss {np.mean(ar_losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    ar_scheduler.step()
    if (epoch + 1) % 5 == 0 :
        if epoch > 5:
            print(eval_model_is(model, ar_model, train_data_loader))
        print(eval_model_oos(model, ar_model, eval_data_loader))
        print(eval_model_is(model, ar_model, eval_data_loader))
        print(eval_ts_model_oos(model, eval_data_loader))

Parameter containing:
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.]], requires_grad=True) torch.Size([45, 51])
torch device is cuda:0
epoch 0


  0%|          | 0/1087 [00:00<?, ?it/s]

epoch 0 avg lstm loss 30.574478149414062, avg ar model loss 220.91648864746094
epoch 0 took 0.5572269320487976 minutes
epoch 1


  0%|          | 0/1087 [00:00<?, ?it/s]

epoch 1 avg lstm loss 0.25716617703437805, avg ar model loss 3.7249555587768555
epoch 1 took 0.556424061457316 minutes
epoch 2


...

epoch 8 avg lstm loss 0.30036720633506775, avg ar model loss -75.2610092163086
epoch 8 took 0.5536255081494649 minutes
epoch 9


  0%|          | 0/1087 [00:00<?, ?it/s]

epoch 9 avg lstm loss 0.2895807921886444, avg ar model loss -75.99514770507812
epoch 9 took 0.5566335956255595 minutes


  0%|          | 0/1087 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [90]:
print(eval_model_is(model, ar_model, test_data_loader))

  0%|          | 0/229 [00:00<?, ?it/s]

[('ME', 0.020979393), ('R2_1', 0.018724356), ('D2P', 0.020316055), ('SPREAD', 0.008505484), ('RVAR', 0.030901603), ('VAR', 0.019735536), ('IdioVol', 0.017966697), ('TURN', 0.017070426), ('SUV', 0.013819655), ('R12_7', 0.0087153455), ('CF2P', 0.015446254), ('E2P', 0.013744907), ('R12_2', 0.008174245), ('S2P', 0.036648627), ('B2M', 0.01940259), ('CF2B', 0.013394699), ('AT', 0.01928784), ('A2ME', 0.023121843), ('Q', 0.026546005), ('PROF', 0.011085588), ('PM', 0.02612281), ('PCM', 0.007775547), ('BETA_m', 0.01722477), ('C2A', 0.21348476), ('OL', 0.03987289), ('ROE', 0.017378375), ('CTO', 0.020674225), ('ATO', 0.022585293), ('LEV', 0.006827993), ('NOA', nan), ('ROA', nan), ('RNA', nan), ('OP', nan), ('NI', nan), ('INV', nan), ('BETA_d', nan), ('R36_13', nan), ('D2A', nan), ('FC2Y', nan), ('SGA2S', nan), ('OA', nan), ('AC', nan), ('HIGH52', nan), ('R60_13', nan), ('DPI2A', nan)]


/tmp/ipykernel_2790/4016302882.py:44: RuntimeWarning: Mean of empty slice
  mse = np.nanmean((preds - gts)**2, axis=0)


In [116]:
print(eval_model_oos(model, ar_model, eval_data_loader), eval_model_is(model, ar_model, train_data_loader))

  0%|          | 0/147 [00:00<?, ?it/s]

  0%|          | 0/1087 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [27]:
def eval_model_is(model, ar_model, data_loader):
    preds = []
    gts = []
    train_masks = []
    for i, data in enumerate(tqdm(data_loader)):
        try:
            data, mask, ordered_mask, factors = data
        except:
            data, mask, ordered_mask, factors, _, _ = data
        assert not torch.isnan(data).any()
    #         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)

        C_train = data.transpose_(0, 1).float().to(device)[:-1]

        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

        factors = factors.transpose_(0, 1).float().to(device)[1:]

        train_input = torch.cat([C_train, C_mask, factors], axis=2)


        if train_input.shape[1] == 10:
            alpha_pred, beta_pred, hidden_out = model(train_input)

            model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
            batch_size, batch_length, dim = model_input.shape
            model_input = model_input.reshape(batch_size * batch_length, dim)

            C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
            C_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
            C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

            pred = ar_model.impute(model_input)
            preds.append(pred.detach().cpu().numpy())
            gts.append(C_train_reshape.detach().cpu().numpy())
            train_masks.append(C_mask_reshape.detach().cpu().numpy())

    preds = np.concatenate(preds, axis=0)
    gts = np.concatenate(gts, axis=0)
    train_masks = np.concatenate(train_masks, axis=0)
    gts[train_masks == 1] = np.nan
    mse = np.nanmean((preds - gts)**2, axis=0)
    return list(zip(chars, np.sqrt(mse)))

def eval_model_oos(model, ar_model, data_loader):
    preds = []
    gts = []
    train_masks = []
    for i, data in enumerate(tqdm(data_loader)):
        data, mask, ordered_mask, factors, oos_data, oos_mask = data
        assert not torch.isnan(data).any()
    #         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)

        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        C_test = oos_data.transpose_(0, 1).float().to(device)[:-1]
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_test_mask = oos_mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

        factors = factors.transpose_(0, 1).float().to(device)[1:]

        train_input = torch.cat([C_train, C_mask, factors], axis=2)


        if train_input.shape[1] == 10:
            alpha_pred, beta_pred, hidden_out = model(train_input)

            model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
            batch_size, batch_length, dim = model_input.shape
            model_input = model_input.reshape(batch_size * batch_length, dim)

            C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
            C_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
            C_test_mask_reshape = C_test_mask[1:].reshape(batch_size * batch_length, 45)
            
            C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)
            C_test_reshape = C_test[1:].reshape(batch_size * batch_length, 45)

            pred = ar_model.impute(model_input)
            preds.append(pred.detach().cpu().numpy())
            gts.append(C_test_reshape.detach().cpu().numpy())
            train_masks.append(C_test_mask_reshape.detach().cpu().numpy())
#             print(preds[-1][train_masks[-1] ==1], gts[-1][train_masks[-1] ==1])
#             print()
            
    preds = np.concatenate(preds, axis=0)
    gts = np.concatenate(gts, axis=0)
    train_masks = np.concatenate(train_masks, axis=0)
    gts[train_masks != 1] = np.nan
    mse = np.nanmean((preds - gts)**2, axis=0)
    return list(zip(chars, np.sqrt(mse)))


def eval_ts_model_is(model, data_loader):
    preds = []
    gts = []
    train_masks = []
    for i, data in enumerate(tqdm(data_loader)):
        try:
            data, mask, ordered_mask, factors = data
        except:
            data, mask, ordered_mask, factors, oos_data, oos_mask  = data
        assert not torch.isnan(data).any()
    #         assert not (data <= 0).any(), torch.min(data)
    #         print(data.shape)
        C_train = data.transpose_(0, 1).float().to(device)[:-1]

        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

        factors = factors.transpose_(0, 1).float().to(device)[1:]

        train_input = torch.cat([C_train, C_mask, factors], axis=2)       
        if train_input.shape[1] == 10:
            alpha_pred, beta_pred,_ = model(train_input)

            pred = alpha_pred / (alpha_pred + beta_pred)
            preds.append(pred.detach().cpu().numpy())
            gts.append(C_train.detach().cpu().numpy())
            train_masks.append(C_mask.detach().cpu().numpy())

    preds = np.concatenate(preds, axis=0)
    gts = np.concatenate(gts, axis=0)
    train_masks = np.concatenate(train_masks, axis=0)
    gts[train_masks == 1] = np.nan

    mse = np.nanmean((preds - gts)**2, axis=(0,1))
    return list(zip(chars[ordering], np.sqrt(mse))), np.mean(np.sqrt(mse))

def eval_ts_model_oos(model, data_loader):
    preds = []
    gts = []
    train_masks = []
    for i, data in enumerate(tqdm(data_loader)):
        data, mask, ordered_mask, factors, oos_data, oos_mask  = data
        assert not torch.isnan(data).any()
    #         assert not (data <= 0).any(), torch.min(data)
    #         print(data.shape)
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        C_test = oos_data.transpose_(0, 1).float().to(device)[:-1]
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        C_test_mask = oos_mask.float().to(device).transpose_(0, 1)[:-1]

        factors = factors.transpose_(0, 1).float().to(device)[1:]

        train_input = torch.cat([C_train, C_mask, factors], axis=2)       
        if train_input.shape[1] == 10:
            alpha_pred, beta_pred,_ = model(train_input)

            pred = alpha_pred / (alpha_pred + beta_pred)
            preds.append(pred.detach().cpu().numpy())
            gts.append(C_test.detach().cpu().numpy())
            train_masks.append(C_test_mask.detach().cpu().numpy())

    preds = np.concatenate(preds, axis=0)
    gts = np.concatenate(gts, axis=0)
    train_masks = np.concatenate(train_masks, axis=0)
    gts[train_masks != 1] = np.nan

    mse = np.nanmean((preds - gts)**2, axis=(0,1))
    return list(zip(chars, np.sqrt(mse))), np.mean(np.sqrt(mse))

  0%|          | 0/147 [00:00<?, ?it/s]

[('PM', 0.27808625),
 ('OP', 0.29084486),
 ('R12_2', 0.28621244),
 ('AC', 0.28896803),
 ('FC2Y', 0.27615285),
 ('DPI2A', 0.27720097),
 ('Q', 0.27766284),
 ('R60_13', 0.288725),
 ('HIGH52', 0.283587),
 ('RNA', 0.29168615),
 ('R12_7', 0.2927738),
 ('B2M', 0.2906263),
 ('ROA', 0.2898613),
 ('SGA2S', 0.29056242),
 ('RVAR', 0.28501433),
 ('SUV', 0.2836168),
 ('D2P', 0.27184334),
 ('ME', 0.2840959),
 ('NOA', 0.2844836),
 ('LEV', 0.28924266),
 ('ATO', 0.28487954),
 ('CTO', 0.28909397),
 ('IdioVol', 0.2876099),
 ('TURN', 0.28521696),
 ('OL', 0.28795233),
 ('D2A', 0.28402916),
 ('CF2P', 0.28753668),
 ('SPREAD', 0.29405928),
 ('PROF', 0.29036552),
 ('BETA_m', 0.28841734),
 ('R36_13', 0.2849772),
 ('BETA_d', 0.286386),
 ('ROE', 0.28672278),
 ('PCM', 0.29475588),
 ('A2ME', 0.28921622),
 ('VAR', 0.29406393),
 ('NI', 0.28846923),
 ('E2P', 0.29119375),
 ('CF2B', 0.28684208),
 ('OA', 0.28919074),
 ('C2A', 0.2905262),
 ('R2_1', 0.28963268),
 ('AT', 0.28462616),
 ('INV', 0.2854173),
 ('S2P', 0.2942026)]

In [ ]:
preds = []
gts = []
train_masks = []
for i, data in enumerate(tqdm(train_data_loader)):
    data, mask, ordered_mask, factors = data
    assert not torch.isnan(data).any()
#         print(data, mask)
    assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)

    C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
    C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
    C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

    factors = factors.transpose_(0, 1).float().to(device)[1:]

    train_input = torch.cat([C_train, C_mask, factors], axis=2)


    if train_input.shape[1] == 10:
        alpha_pred, beta_pred, hidden_out = model(train_input)

        model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
        batch_size, batch_length, dim = model_input.shape
        model_input = model_input.reshape(batch_size * batch_length, dim)

        C_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
        C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_size * batch_length, 45)
        C_train_reshape = C_train[1:].reshape(batch_size * batch_length, 45)

        alphas_pred, betas_pred = ar_model(model_input)
        pred = alphas_pred / (alphas_pred + betas_pred)
        preds.append(pred.detach().cpu().numpy())
        gts.append(C_train_reshape.detach().cpu().numpy())
        train_masks.append(C_ordered_mask_reshape.detach().cpu().numpy())

preds = np.concatenate(preds, axis=0)
gts = np.concatenate(gts, axis=0)
train_masks = np.concatenate(train_masks, axis=0)
gts[train_masks == 1] = np.nan
mse = np.nanmean((preds - gts)**2, axis=0)
list(zip(chars[ordering], np.sqrt(mse)))